In [12]:
# 导入包和模块
import torch as t
import torchvision 
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt
import time 
import sys
from d2lzh_pytorch import *

# 载入FashionMNIST数据集
mnist_train = torchvision.datasets.FashionMNIST(root='~/Datasets/MNIST', train=True,
                                                download=False, transform=transforms.ToTensor())
mnist_test = torchvision.datasets.FashionMNIST(root='~/Datasets/MNIST', train=False,
                                               download=False, transform=transforms.ToTensor())
print(type(mnist_train))
print(len(mnist_train), len(mnist_test))


<class 'torchvision.datasets.mnist.FashionMNIST'>
60000 10000


In [6]:

# feature, label = mnist_train[0]
# print(feature.shape, label)
# 
# X,y = [], []
# for i in range(10):
#     X.append(mnist_train[i][0])
#     y.append(mnist_train[i][1])
#     print(y)
#     # print(get_fashion_mnist_labels(y))
# show_fashion_mnist(X, get_fashion_mnist_labels(y))


In [7]:
# 读取小批量

batch_size = 256
if sys.platform.startswith('win'):
    num_worker = 0
else:
    num_worker = 4
    
train_iter = t.utils.data.DataLoader(mnist_train, batch_size=batch_size, shuffle=True,
                                     num_workers=num_worker)
test_iter = t.utils.data.DataLoader(mnist_test, batch_size=batch_size, shuffle=False,
                                    num_workers = num_worker)

# start =time.time()
# for X,y in train_iter:
#     continue
# print("%.2f sec" %(time.time()-start))


In [13]:
#初始化 模型参数
num_inputs = 784
num_outputs = 10
W = t.tensor(np.random.normal(0, 0.01, (num_inputs, num_outputs)),
             dtype=t.float,requires_grad=True)
b = t.zeros(num_outputs, dtype=t.float, requires_grad=True)

# 实现softmax运算
def softmax(X):
    X_exp = X.exp();
    partition = X_exp.sum(dim=1, keepdim=True)
    return X_exp/partition

# 定义模型
def net(X):
    return softmax(t.mm(X.view((-1, num_inputs)), W)+b)

# loss

# y_hat = t.tensor([[0.1, 0.3, 0.6], [0.3, 0.2, 0.5]])
# y = t.LongTensor([0, 2])
# # y_hat.gather(1, y.view(-1, 1))
# print(y_hat.gather(1, y.view(-1, 1)))


def cross_entropy(y_hat, y):
    return -t.log(y_hat.gather(1, y.view(-1,1)))


def accuracy(y_hat, y):
    return (y_hat.argmax(dim=1) == y).float().mean().item()


def evaluate_accuracy(data_iter, net):
    acc_sum, n = 0.0, 0
    for X, y in data_iter:
        acc_sum += (net(X).argmax(dim=1) == y).float().sum().item()
        n+=y.shape[0]
        
    return acc_sum/n

num_epochs, lr = 5, 0.1

def train_ch3(net, train_iter, test_iter, loss, num_epochs, batch_size,
              params=None, lr=None, optimizer=None):
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n = 0.0, 0.0, 0
        for X ,y in train_iter:
            y_hat = net(X)
            l = loss(y_hat, y).sum()
            
            # 梯度清零
            if optimizer is not None:
                optimizer.zero_grad()
            elif params is not None and params[0].grad is not None:
                for param in params:
                    param.grad.data.zero_()
                    
            l.backward()
            if optimizer is None:
                sgd(params, lr, batch_size)
            else:
                optimizer.step()
                
            train_l_sum += l.item()
            train_acc_sum += (y_hat.argmax(dim=1) ==y).sum().item()
            n+=y.shape[0]
            
        test_acc = evaluate_accuracy(test_iter, net)
        print('epoch %d, loss %.4f, train acc %.3f, test acc %.3f' %(epoch+1, train_l_sum/n, train_acc_sum/n, test_acc))
        
train_ch3(net, train_iter, test_iter, cross_entropy, num_epochs, batch_size, [W,b], lr)

            
            

epoch 5, loss 0.4851, train acc 0.837, test acc 0.827
